In [1]:
!pip install xgboost
!pip install datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Load the GoEmotions dataset
dataset = load_dataset("go_emotions")

# Simplify the dataset by filtering out examples with multiple labels
def simplify(example):
    return len(example['labels']) == 1

dataset = dataset.filter(simplify)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Filter:   0%|          | 0/43410 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5426 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [3]:
# Extract text and single-label categories
def preprocess_labels(example):
    example['label'] = example['labels'][0]  # Take the single label
    return example

dataset = dataset.map(preprocess_labels)
dataset = dataset.remove_columns(['labels'])

Map:   0%|          | 0/36308 [00:00<?, ? examples/s]

Map:   0%|          | 0/4548 [00:00<?, ? examples/s]

Map:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizing the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=2, max_df=0.95      # Ignore rare and very common words
)
X_train = vectorizer.fit_transform(dataset['train']['text'])
X_val = vectorizer.transform(dataset['validation']['text'])
X_test = vectorizer.transform(dataset['test']['text'])

y_train = dataset['train']['label']
y_val = dataset['validation']['label']
y_test = dataset['test']['label']

In [5]:
# feature scaling
from sklearn.preprocessing import MaxAbsScaler

# Apply MaxAbsScaler to normalize features
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [7]:
# Initialize XGBoost classifier
xgb_model = XGBClassifier(
    objective='multi:softmax',  # Multi-class classification with softmax
    num_class=len(set(y_train)),  # Number of classes
    n_jobs=-1,
    use_label_encoder=False,     # Disable label encoder to avoid warnings
    eval_metric='mlogloss',      # Evaluation metric for multi-class classification
    random_state=42              # For reproducibility
)

# Train the model
xgb_model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:30:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=-1, num_class=28, num_parallel_tree=None, ...)

In [8]:
# Predict on training, validation, and test sets
y_train_pred_xgb = xgb_model.predict(X_train_scaled)
y_val_pred_xgb = xgb_model.predict(X_val_scaled)
y_test_pred_xgb = xgb_model.predict(X_test_scaled)

# Calculate accuracy
train_accuracy_xgb = accuracy_score(y_train, y_train_pred_xgb)
val_accuracy_xgb = accuracy_score(y_val, y_val_pred_xgb)
test_accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)

# Print results
print(f"XGBoost Training Accuracy: {train_accuracy_xgb:.4f}")
print(f"XGBoost Validation Accuracy: {val_accuracy_xgb:.4f}")
print(f"XGBoost Test Accuracy: {test_accuracy_xgb:.4f}")

# Classification report
print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred_xgb))

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred_xgb))

XGBoost Training Accuracy: 0.7259
XGBoost Validation Accuracy: 0.5778
XGBoost Test Accuracy: 0.5725

Classification Report (Validation Set):
              precision    recall  f1-score   support

           0       0.66      0.62      0.64       326
           1       0.73      0.79      0.76       208
           2       0.40      0.31      0.35       109
           3       0.43      0.11      0.17       164
           4       0.56      0.15      0.24       258
           5       0.54      0.15      0.23        96
           6       0.57      0.20      0.29       102
           7       0.55      0.20      0.29       164
           8       0.69      0.38      0.49        52
           9       0.60      0.07      0.12        91
          10       0.30      0.07      0.11       212
          11       0.56      0.41      0.47        61
          12       0.70      0.35      0.47        20
          13       0.60      0.17      0.27        52
          14       0.68      0.47      0.55     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint  # For continuous and discrete distributions

# Define parameter distributions
param_distributions_xgb = {
    'max_depth': randint(3, 11),        # Depth of trees: Randomly choose between 3 and 10
    'learning_rate': uniform(0.01, 0.2), # Learning rate: Randomly sample between 0.01 and 0.21
    'n_estimators': randint(50, 201),    # Boosting rounds: Randomly choose between 50 and 200
    'subsample': uniform(0.8, 0.2),      # Fraction of data: Randomly sample between 0.8 and 1.0
    'colsample_bytree': uniform(0.8, 0.2) # Features fraction: Randomly sample between 0.8 and 1.0
}

# Perform randomized search
random_search_xgb = RandomizedSearchCV(
    XGBClassifier(objective='multi:softmax', num_class=len(set(y_train)), use_label_encoder=False, random_state=42),
    param_distributions=param_distributions_xgb,
    n_iter=20,                 # Number of parameter settings to sample
    scoring='accuracy',        # Scoring metric
    cv=3,                      # 3-fold cross-validation
    verbose=1,                 # Show progress
    random_state=42,           # For reproducibility
    n_jobs=-1                  # Use all available cores
)

random_search_xgb.fit(X_train_scaled, y_train)

# Display best parameters and accuracy
print(f"Best Parameters: {random_search_xgb.best_params_}")
print(f"Best Cross-Validation Accuracy: {random_search_xgb.best_score_:.4f}")

# Use the best model
best_xgb_model = random_search_xgb.best_estimator_

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:23:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.878976303635114, 'learning_rate': 0.06869763494360763, 'max_depth': 9, 'n_estimators': 188, 'subsample': 0.8397684808177611}
Best Cross-Validation Accuracy: 0.5627


Best Parameters: {'colsample_bytree': 0.878976303635114, 'learning_rate': 0.06869763494360763, 'max_depth': 9, 'n_estimators': 188, 'subsample': 0.8397684808177611}


In [9]:
best_xgb_model = XGBClassifier(
    objective='multi:softmax',  # Multi-class classification with softmax
    num_class=len(set(y_train)),  # Number of classes
    n_jobs=-1,
    use_label_encoder=False,     # Disable label encoder to avoid warnings
    eval_metric='mlogloss',      # Evaluation metric for multi-class classification
    random_state=42,              # For reproducibility
    colsample_bytree=0.88,
    learning_rate=0.069,
    max_depth=9,
    n_estimators=188,
    subsample=0.84
)
best_xgb_model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:45:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.88, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.069, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=188,
              n_jobs=-1, num_class=28, num_parallel_tree=None, ...)

In [10]:
# Predict on all datasets
y_train_pred = best_xgb_model.predict(X_train_scaled)
y_val_pred = best_xgb_model.predict(X_val_scaled)
y_test_pred = best_xgb_model.predict(X_test_scaled)

# Calculate accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the accuracies
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Detailed classification reports
print("\nClassification Report (Training Set):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred))

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred))

Training Accuracy: 0.7322
Validation Accuracy: 0.5787
Test Accuracy: 0.5719

Classification Report (Training Set):
              precision    recall  f1-score   support

           0       0.82      0.71      0.76      2710
           1       0.85      0.88      0.86      1652
           2       0.82      0.53      0.64      1025
           3       0.96      0.29      0.44      1451
           4       0.93      0.30      0.46      1873
           5       0.96      0.55      0.70       649
           6       0.97      0.53      0.69       858
           7       0.95      0.41      0.58      1389
           8       0.94      0.78      0.86       389
           9       0.99      0.34      0.51       709
          10       0.94      0.37      0.53      1402
          11       0.92      0.63      0.75       498
          12       0.98      0.62      0.76       203
          13       0.89      0.44      0.59       510
          14       0.94      0.73      0.82       430
          15       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
y_train_pred = best_xgb_model.predict(X_train_scaled)
y_val_pred = best_xgb_model.predict(X_val_scaled)
y_test_pred = best_xgb_model.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 0.7311
Validation Accuracy: 0.5778
Test Accuracy: 0.5702
